In [1]:
import site
site.addsitedir('Lib/site-packages')
from dotenv import load_dotenv
load_dotenv()
import getpass
import os
import requests
import ffmpeg
import json
import time
#import asyncio
from pydantic import BaseModel, Field
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode
from lumaai import LumaAI
from typing import Literal

In [2]:
#LUMA
client = LumaAI(
    auth_token=os.environ.get("LUMAAI_API_KEY"),
)
#OPENAI
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model
#LANGCHAIN
llm = init_chat_model("gpt-4o-mini", model_provider="openai")

## SCHEMAS

In [4]:
class video_gen_schema(BaseModel):
    """Generates a video using text input and returns a filepath to the video"""

    vid_prompt: str = Field(..., description="The textual prompt used in generating the video")
    vid_duration: bool = Field(..., description="The duration of the video to generate. A value of FALSE means 5 seconds and a value of TRUE means 9 seconds")

class step_by_step_output_schema(BaseModel):
    """Prints a storyboard for the user to view in string format"""
    story_description: str = Field(..., description="A 200-250 word description of the story")
    character_details: str = Field(..., description="A 100-200 word description of each character in the scene")
    background_details: str = Field(..., description="A 50-100 description of the scene")
    auditory_details: str = Field(..., description="A 50-100 description of the voice profile of each character")
    dialogue_details: str = Field(..., description="The parts of the dialogue that require enunciation and emotion")

## TOOLS

In [6]:
@tool("video_gen_tool",args_schema=video_gen_schema)   
def generate_vid(vid_prompt: str, vid_duration: bool) -> str:
    dur = "5s"
    if(vid_duration):
        dur = "9s"
    generation = client.generations.create(
        prompt=vid_prompt,
        model="ray-2",
        resolution="720p",
        duration=dur
    )
    completed = False
    while not completed:
      generation = client.generations.get(id=generation.id)
      if generation.state == "completed":
        completed = True
      elif generation.state == "failed":
        raise RuntimeError(f"Generation failed: {generation.failure_reason}")
      print("Dreaming")
      time.sleep(5)
    video_url = generation.assets.video
    # download the video
    response = requests.get(video_url, stream=True)
    with open(f'{generation.id}.mp4', 'wb') as file:
        file.write(response.content)
    print(f"File downloaded as {generation.id}.mp4")
@tool("storyboard_tool",args_schema=step_by_step_output_schema)
def generate_storyboard(story_description: str,character_details: str,background_details: str,auditory_details: str,dialogue_details: str) -> str:
    temp_storyboard = "STORY ANALYSIS\n------------------------------\n"+story_description+"\nCHARACTERS\n------------------------------\n"+character_details
    temp_storyboard+="\nBACKGROUND\n------------------------------\n"+background_details+"\nAUDIO\n------------------------------\n"+auditory_details+"\nDIALOGUE\n------------------------------\n"+dialogue_details
    print(temp_storyboard)
    return temp_storyboard
#########################################CODE ABOVE THIS
## BINDING
tools = [generate_vid, generate_storyboard]

tool_node = ToolNode(tools)

model = llm.bind_tools(tools)

## RUNNING MODEL

In [7]:
# Define the function that determines whether to continue or not
def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "tools"
    # Otherwise, we stop (reply to the user)
    return END


# Define the function that calls the model
def call_model(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}


# Define a new graph
workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.add_edge(START, "agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("tools", 'agent')

# Initialize memory to persist state between graph runs
checkpointer = MemorySaver()

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable.
# Note that we're (optionally) passing the memory when compiling the graph
app = workflow.compile(checkpointer=checkpointer)

# Use the agent
final_state = app.invoke(
    {"messages": [{"role": "user", "content": "Generate a storyboard and corresponding video about the following scene: INT. MONTAGUE MUSEUM – MODERN DAYThe once-grand family home, now a tour-driven Hearst Castle-like museum.ELIZABETH MONTAGUE, intense and angry, argues with Ethan.ELIZABETHYou know you can’t tell a soul,right?ETHANAre you kidding? I’m going to telleveryone. You can’t hide this."}]},
    config={"configurable": {"thread_id": 42}}
)
final_state["messages"][-1].content

STORY ANALYSIS
------------------------------
The scene unfolds in the Montague Museum, a once-grand family home now transformed into a lavish museum reminiscent of Hearst Castle. The opulent surroundings contrast sharply with the emotional intensity of the argument between Elizabeth Montague and Ethan. Elizabeth, a woman in her late thirties with fierce determination, confronts Ethan, a younger man with an air of rebelliousness. As the tension rises, the echoes of their confrontation resonate in the grand hall adorned with historical artifacts, which serve as silent witnesses to their heated exchange.
CHARACTERS
------------------------------
ELIZABETH MONTAGUE: In her late thirties, Elizabeth has piercing blue eyes and shoulder-length dark hair, often styled in an elegant yet assertive manner. She dresses conservatively but stylishly, reflecting her status. Her expression is often intense, filled with anger and frustration.  ETHAN: A younger man in his mid-twenties with tousled hair 

'### Storyboard Overview\n\n**Scene Description:**\nThe scene takes place in the Montague Museum, a luxurious and historically rich setting that is a transformed family estate reminiscent of Hearst Castle. The emotional conflict is palpable as Elizabeth Montague confronts Ethan about a secret that must not be disclosed, their argument echoing through the grand halls adorned with exquisite artifacts.\n\n---\n\n**Characters:**\n\n- **Elizabeth Montague:** \n  - Late thirties, intense demeanor.\n  - Piercing blue eyes and shoulder-length dark hair. \n  - Dresses conservatively yet stylishly, reflecting her status.\n  - Her expressions are often filled with anger and frustration, speaking sharply.\n\n- **Ethan:**\n  - Mid-twenties, casual and laid-back, with tousled hair.\n  - Dresses in casual attire, embodying youthful rebellion.\n  - Possesses undeniable charm and often wears a sly grin, even when being defiant.\n\n---\n\n**Background Details:**\nThe Montague Museum is opulently decorat